In [4]:
import dotenv

dotenv.load_dotenv()

True

In [23]:
from langchain_community.document_loaders import PyPDFium2Loader
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.embeddings import CacheBackedEmbeddings
from langchain_core.prompts import PromptTemplate





In [8]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [24]:
# Load, chunk and index the contents of pdf files
loader = PyPDFium2Loader("django-readthedocs-io-en-latest.pdf")
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
embeddings = OpenAIEmbeddings()
store = LocalFileStore("./cache/")
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    embeddings, store, namespace=embeddings.model
)
vector_store = FAISS.from_documents(documents, cached_embedder)
retriever = vector_store.as_retriever()

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)


In [25]:
for chunk in rag_chain.stream("Explain the Django admin interface."):
    print(chunk, end="", flush=True)

The Django admin interface is an automated feature that generates admin sites for managing content without needing much creativity. It allows site managers to add, change, and delete content easily. The admin interface is not intended for site visitors but rather for site managers to edit content. Thanks for asking!